In [ ]:
#Algunos datos de utilidad según conclusiones del TP1

#En qué horarios se realizan más subastas?
# Teniendo en cuenta solo los días hábiles
# 5~10    : < 200k
# 4,11~20 : 200k~300k
# 20~0   : 300k ~ 400k
# 1~2    : > 400k

# :. entre las 20~2 hs se realizan la mayor cantidad de subastas con picos entre 1~2 hs.

In [ ]:
# Depende un evento o una conversión de un usuario del día de la semana? Y de la hora?

#La cantidad de eventos se mantine constantes, excepto en los días M y X, donde encontramos un pico.
#Teniendo en cuenta la hora, la mayor cantidad de eventos se produce durante la tarde noche, inicio de la madrugada:
#Pico(+175k):    [0,1]
#Alta(150k~100k):(1,4)U[15,0)
#Media(100k~50k): [4,15)
#Baja(0~25k):(7,9)
#:. La mayor cantidad de eventos sucede durante el rango 15~1(tarde noche)

#INSTALACIONES
# La mayor cantidad se realiza los días M y X
# Los días M y X se duplica la cantidad de instalaciones aproximadamente(350~400)vs(750~800)
# Respecto del horario: Comportamiento similar a los eventos; la mayor cantidad se registran durante la tarde~noche(13~3)
# Afecta la presencia de wifi a la cantidad de instalaciones?
#El 80% de las instalaciones se lleva a cabo con dispositivos usando wifi.
#Cualés son las app mas populares?
# app_id | users | events
# 66       70k      325k
# 63       18k      181k 
# 145      17.5k    252k
# 64       17k      259k
# 103       15k     137k

## Predicciones
- En un instante dado, estimar St(d) el tiempo hasta que un dispositivo d aparezca de vuelta en una subasta RTB
- En un instante dado, estimar Sc(d) el tiempo hasta que un dispositivo d convierta



# Arboles - xgboost

In [110]:
import pandas as pd
import matplotlib as mp
import numpy as np
from datetime import timedelta
import datetime as dt
import gc

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import xgboost as xgb

%matplotlib inline

## Preparación de datos

## Configuracion

In [19]:
#Root de archivos de datos
root_dir = '../data/'
print('Root path data: '+root_dir)
#Lee menos data si es testing
is_testing = True
print('Es testing: '+str(is_testing))
fraccion_a_usar = 0.01
print('Faccion de registros a usar para testing: '+str(fraccion_a_usar))
# Cantidad minima de aparicion de equipos en un dataframe. Esta variable se usa para eliminar los registros
# que tengan pocas apariciones dentro del set de datos total.
min_devices = 10
print('Min de device a tener en cuenta: '+str(min_devices))

# Identificador unico de equipos
device_uuid = ['ref_type','ref_hash']

# Ventanas de tiempo
windows = pd.DataFrame({
'window_nr':[1,2,3,4],
'n': 1,
'min_date': dt.datetime(2019,4,18),
'max_date': [dt.datetime(2019,4,21),dt.datetime(2019,4,22),dt.datetime(2019,4,23),dt.datetime(2019,4,24)]
})

Root path data: ../data/
Es testing: True
Faccion de registros a usar para testing: 0.01
Min de device a tener en cuenta: 10


In [20]:
windows

,max_date,min_date,n,window_nr
0,2019-04-21,2019-04-18,1,1
1,2019-04-22,2019-04-18,1,2
2,2019-04-23,2019-04-18,1,3
3,2019-04-24,2019-04-18,1,4


In [4]:
#Se lee el dataframe de auctions

#Optimizado para menos memoria
auction_dtypes = {
    'ref_type_id': np.int8,
    'device_id': np.int64,
    'source_id': np.int8
}
#root_dir + 'auctions.csv.gzip'
auctions = pd.read_csv(root_dir+'auctions.csv.gzip',
                       compression = 'gzip',
                       dtype = auction_dtypes,
                       parse_dates = ['date'])
auctions.rename({'device_id':'ref_hash',
                 'ref_type_id':'ref_type'}, inplace=True, axis='columns')

# Para sumar los registros
auctions['n'] = 1

In [5]:
# Info de registros totales/a usar para entrenar
print('Total records:',auctions['n'].sum())
if is_testing:
    auctions = auctions.sample(frac=fraccion_a_usar)
    print("Is Testing, #records:", auctions['n'].sum())
else:
    print("Not Testing, #records:", auctions['n'].sum())

('Total records:', 47409528)
('Is Testing, #records:', 474095)


In [7]:
to_predict = pd.read_csv(root_dir+'target_competencia_ids.csv')
#predict reaparicion=st
predict_ids = to_predict.apply(lambda x: np.int64(x['ref_hash'][0:x['ref_hash'].find('_')]), axis='columns').drop_duplicates().to_frame()
#predict conversion=sc
predict_ids.head()

,0
0,1000169251625791246
2,1000395625957344683
4,1003027494996471685
6,1006670001679961544
8,1007573308966476713


### Arreglo de datos en auctions

In [8]:
auctions.head(3)

,date,ref_hash,ref_type,source_id,n
26464112,2019-04-21 02:25:26.903358,6489924860691406939,1,1,1
41622451,2019-04-22 22:11:57.903062,3422777417513500532,7,0,1
43335676,2019-04-21 03:16:02.192333,1458201828341030914,7,0,1


In [9]:
# Se eliminan los reg con pocas apariciones dentro del set de datos
print('Minimo de devices a usar '+str(min_devices))
orig_count = auctions['n'].sum()
auctions_clean = auctions.groupby(device_uuid, sort = False).filter(lambda x: x['n'].sum() >= min_devices) 
last_count = auctions_clean['n'].sum()
print('Cantidad auctions originales '+str(orig_count))
print('Cantidad auctions eliminados '+str(orig_count-last_count))
print('Cantidad auctions restantes '+str(last_count))

Minimo de devices a usar 10
Cantidad auctions originales 474095
Cantidad auctions eliminados 288706
Cantidad auctions finales 185389


In [10]:
auctions_clean.head()

,date,ref_hash,ref_type,source_id,n
43335676,2019-04-21 03:16:02.192333,1458201828341030914,7,0,1
24741188,2019-04-25 17:24:40.246125,1260686783384403796,1,1,1
26783913,2019-04-25 21:11:05.549285,3424623430150500455,1,1,1
28617588,2019-04-25 21:25:26.432189,3086216490891530850,7,1,1
35126626,2019-04-25 14:26:09.479632,8324366947559667326,1,1,1


In [18]:
device_uuid+['date']

['ref_type', 'ref_hash', 'date']

In [22]:
auctions_clean.dtypes

date         datetime64[ns]
ref_hash              int64
ref_type               int8
source_id              int8
n                     int64
dtype: object

In [12]:
auc_clean = auctions_clean.sort_values(by=device_uuid+['date'], inplace=False)
#setea el próximo día de aparición(next_date)
auc_clean['next_date'] = auc_clean.groupby(device_uuid, as_index = False, sort=False)['date'].transform(lambda x: x.shift(-1))
# saco los registros 'borde', aquellos que no tienen 'next_date'
auc_clean = auc_clean.loc[(~auc_clean['next_date'].isnull())]
# Seteo los segundos de las apariciones
auc_clean['secs_to_next'] = (auc_clean['next_date'] - auc_clean['date']).transform(lambda x: round(x.total_seconds()))
auc_clean.head()

,date,ref_hash,ref_type,source_id,n,next_date,secs_to_next
19683348,2019-04-18 07:18:46.350778,2125435237741841,1,3,1,2019-04-18 07:22:18.798585,212.0
4895991,2019-04-18 07:22:18.798585,2125435237741841,1,1,1,2019-04-18 08:35:32.924509,4394.0
9278379,2019-04-18 08:35:32.924509,2125435237741841,1,3,1,2019-04-18 08:54:54.872365,1162.0
41311212,2019-04-18 08:54:54.872365,2125435237741841,1,3,1,2019-04-18 20:48:18.888273,42804.0
13938745,2019-04-18 20:48:18.888273,2125435237741841,1,3,1,2019-04-19 14:15:39.599201,62841.0


In [13]:
auc_clean['hs_to_next'] = (auc_clean['secs_to_next'] ).transform(lambda x: x/(60*60))
auc_clean.head()

,date,ref_hash,ref_type,source_id,n,next_date,secs_to_next,hs_to_next
19683348,2019-04-18 07:18:46.350778,2125435237741841,1,3,1,2019-04-18 07:22:18.798585,212.0,0.058889
4895991,2019-04-18 07:22:18.798585,2125435237741841,1,1,1,2019-04-18 08:35:32.924509,4394.0,1.220556
9278379,2019-04-18 08:35:32.924509,2125435237741841,1,3,1,2019-04-18 08:54:54.872365,1162.0,0.322778
41311212,2019-04-18 08:54:54.872365,2125435237741841,1,3,1,2019-04-18 20:48:18.888273,42804.0,11.890000
13938745,2019-04-18 20:48:18.888273,2125435237741841,1,3,1,2019-04-19 14:15:39.599201,62841.0,17.455833


In [14]:
auc_clean['n'].sum()

176106

In [15]:
auc_clean['date'].min()

Timestamp('2019-04-18 00:00:00.259283')

In [16]:
auc_clean['date'].max()

Timestamp('2019-04-26 23:57:42.603836')

# Armado de features

In [22]:
windows.set_index('window_nr')

,max_date,min_date,n
window_nr,,,
1,2019-04-21,2019-04-18,1
2,2019-04-22,2019-04-18,1
3,2019-04-23,2019-04-18,1
4,2019-04-24,2019-04-18,1


## Rearmado de ventanas

In [25]:
windows2 = pd.DataFrame({
'window_nr':[1,2,3,4],
'n': 1,
'min_date': [dt.datetime(2019,4,18),dt.datetime(2019,4,19),dt.datetime(2019,4,20),dt.datetime(2019,4,21)],
'max_date': [dt.datetime(2019,4,21),dt.datetime(2019,4,22),dt.datetime(2019,4,23),dt.datetime(2019,4,24)]
})
windows2

,max_date,min_date,n,window_nr
0,2019-04-21,2019-04-18,1,1
1,2019-04-22,2019-04-19,1,2
2,2019-04-23,2019-04-20,1,3
3,2019-04-24,2019-04-21,1,4


In [26]:
cutoff_date = windows2.set_index('window_nr').loc[1,'max_date']
cutoff_date

Timestamp('2019-04-21 00:00:00')

In [27]:
train_auctions = auc_clean.loc[auctions['date'] < cutoff_date]
train_auctions.head(3)
train_auctions.count()

date            60778
ref_hash        60778
ref_type        60778
source_id       60778
n               60778
next_date       60778
secs_to_next    60778
hs_to_next      60778
dtype: int64

In [28]:
device_uuid

['ref_type', 'ref_hash']

In [29]:
#Veo cuantos device tengo para el 'train_auctions'
devices = train_auctions[device_uuid].drop_duplicates()
devices['ref_hash'].count()

8066

In [99]:
#train_auctions[train_auctions['ref_hash'] == 2125435237741841]#13
train_auctions.groupby(by='ref_hash')['secs_to_next'].sum().loc[2125435237741841,]/13#344042.0

26464.76923076923

In [37]:
#train_auctions2 = train_auctions[[]]
train_auc = train_auc.reset_index()
train_auc = train_auc[['date','ref_hash','source_id','next_date','secs_to_next','hs_to_next']]
train_auc.head()

,date,ref_hash,source_id,next_date,secs_to_next,hs_to_next
0,2019-04-18 07:18:46.350778,2125435237741841,3,2019-04-18 07:22:18.798585,212.0,0.058889
1,2019-04-18 07:22:18.798585,2125435237741841,1,2019-04-18 08:35:32.924509,4394.0,1.220556
2,2019-04-18 08:35:32.924509,2125435237741841,3,2019-04-18 08:54:54.872365,1162.0,0.322778
3,2019-04-18 08:54:54.872365,2125435237741841,3,2019-04-18 20:48:18.888273,42804.0,11.890000
4,2019-04-18 20:48:18.888273,2125435237741841,3,2019-04-19 14:15:39.599201,62841.0,17.455833


In [38]:
#Se desea saber el tiempo promedio entre arribos de los dispositivos a las encuestas.
temp = train_auctions.groupby(device_uuid, as_index='False')['secs_to_next'].mean().to_frame()['secs_to_next']
temp.columns = ['secs_to_next_mean']

In [55]:
temp=train_auctions.groupby(device_uuid, as_index='False')['secs_to_next'].mean().to_frame().reset_index()
temp=temp.rename(columns={'secs_to_next':'secs_to_next_mean'})
temp.head()

,ref_type,ref_hash,secs_to_next_mean
0,1,2125435237741841,40927.750000
1,1,2417786714126835,59362.166667
2,1,3918731566612368,37712.000000
3,1,4542345673503215,82628.333333
4,1,6380906313550611,54287.200000


In [57]:
train_auc2 = train_auc.merge(temp,how='inner',on='ref_hash')
train_auc2.head()

,date,ref_hash,source_id,next_date,secs_to_next,hs_to_next,ref_type,secs_to_next_mean
0,2019-04-18 07:18:46.350778,2125435237741841,3,2019-04-18 07:22:18.798585,212.0,0.058889,1,40927.75
1,2019-04-18 07:22:18.798585,2125435237741841,1,2019-04-18 08:35:32.924509,4394.0,1.220556,1,40927.75
2,2019-04-18 08:35:32.924509,2125435237741841,3,2019-04-18 08:54:54.872365,1162.0,0.322778,1,40927.75
3,2019-04-18 08:54:54.872365,2125435237741841,3,2019-04-18 20:48:18.888273,42804.0,11.890000,1,40927.75
4,2019-04-18 20:48:18.888273,2125435237741841,3,2019-04-19 14:15:39.599201,62841.0,17.455833,1,40927.75


# Primer prueba con XGBoost

In [58]:
windows2

,max_date,min_date,n,window_nr
0,2019-04-21,2019-04-18,1,1
1,2019-04-22,2019-04-19,1,2
2,2019-04-23,2019-04-20,1,3
3,2019-04-24,2019-04-21,1,4


In [65]:
win = 0
max_date = windows2.loc[win,].max_date

In [71]:
#Filtramos para la primer ventana
train_auc2 = train_auc2.groupby('date').filter(lambda x: x.date < max_date)
train_auc2.head()

,date,ref_hash,source_id,next_date,secs_to_next,hs_to_next,ref_type,secs_to_next_mean
0,2019-04-18 07:18:46.350778,2125435237741841,3,2019-04-18 07:22:18.798585,212.0,0.058889,1,40927.75
1,2019-04-18 07:22:18.798585,2125435237741841,1,2019-04-18 08:35:32.924509,4394.0,1.220556,1,40927.75
2,2019-04-18 08:35:32.924509,2125435237741841,3,2019-04-18 08:54:54.872365,1162.0,0.322778,1,40927.75
3,2019-04-18 08:54:54.872365,2125435237741841,3,2019-04-18 20:48:18.888273,42804.0,11.890000,1,40927.75
4,2019-04-18 20:48:18.888273,2125435237741841,3,2019-04-19 14:15:39.599201,62841.0,17.455833,1,40927.75


In [132]:
#convertidor de fecha(Timestamp) a int
def convert_date_int(date):
    d = pd.to_datetime(date)
    return np.datetime64(d).astype(int)

In [133]:
convert_date_int(d)

1555545600259283

In [138]:
train_auc2['date_int'] = train_auc2['date'].apply(convert_date_int)
train_auc2['next_date_int'] = train_auc2['next_date'].apply(convert_date_int)

In [139]:
train_auc2.head()

,date,ref_hash,source_id,next_date,secs_to_next,hs_to_next,ref_type,secs_to_next_mean,date_int,next_date_int
0,2019-04-18 07:18:46.350778,2125435237741841,3,2019-04-18 07:22:18.798585,212.0,0.058889,1,40927.75,1555571926350778,1555572138798585
1,2019-04-18 07:22:18.798585,2125435237741841,1,2019-04-18 08:35:32.924509,4394.0,1.220556,1,40927.75,1555572138798585,1555576532924509
2,2019-04-18 08:35:32.924509,2125435237741841,3,2019-04-18 08:54:54.872365,1162.0,0.322778,1,40927.75,1555576532924509,1555577694872365
3,2019-04-18 08:54:54.872365,2125435237741841,3,2019-04-18 20:48:18.888273,42804.0,11.890000,1,40927.75,1555577694872365,1555620498888273
4,2019-04-18 20:48:18.888273,2125435237741841,3,2019-04-19 14:15:39.599201,62841.0,17.455833,1,40927.75,1555620498888273,1555683339599201


In [143]:
columns_to_xgboost = ['ref_hash','source_id','secs_to_next',\
                      'hs_to_next','ref_type','secs_to_next_mean'\
                      ,'date_int','next_date_int']

In [144]:
train_auc3 = train_auc2[columns_to_xgboost]

In [145]:
train_auc3.head()

,ref_hash,source_id,secs_to_next,hs_to_next,ref_type,secs_to_next_mean,date_int,next_date_int
0,2125435237741841,3,212.0,0.058889,1,40927.75,1555571926350778,1555572138798585
1,2125435237741841,1,4394.0,1.220556,1,40927.75,1555572138798585,1555576532924509
2,2125435237741841,3,1162.0,0.322778,1,40927.75,1555576532924509,1555577694872365
3,2125435237741841,3,42804.0,11.890000,1,40927.75,1555577694872365,1555620498888273
4,2125435237741841,3,62841.0,17.455833,1,40927.75,1555620498888273,1555683339599201


In [148]:
#Separamos X y Y
X = train_auc3.loc[:,train_auc3.columns != 'next_date_int']

In [149]:
X.head()

,ref_hash,source_id,secs_to_next,hs_to_next,ref_type,secs_to_next_mean,date_int
0,2125435237741841,3,212.0,0.058889,1,40927.75,1555571926350778
1,2125435237741841,1,4394.0,1.220556,1,40927.75,1555572138798585
2,2125435237741841,3,1162.0,0.322778,1,40927.75,1555576532924509
3,2125435237741841,3,42804.0,11.890000,1,40927.75,1555577694872365
4,2125435237741841,3,62841.0,17.455833,1,40927.75,1555620498888273


In [151]:
y = train_auc3.loc[:,train_auc3.columns == 'next_date_int']
y.head()

,next_date_int
0,1555572138798585
1,1555576532924509
2,1555577694872365
3,1555620498888273
4,1555683339599201


In [152]:
train_auc3.shape,X.shape,y.shape

((60778, 8), (60778, 7), (60778, 1))

In [153]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=1)

### Hiper-parámetros
- learning_rate: tasa de aprendizaje
- max_depth: máxima profundidad de cada árbol(default=6)
- subsample: porcentaje de muestras usadas para cada árbol (valor muy bajo, posible underfitting)
- colsample_bytree: porcentaje de features usadas para cada árbol (valores muy alto, posible overfitting)
- n_estimators: cantidad de árboles a construir.
- objective: función de error a utilizar (algunas: reg:linear para regresión, reg:logistic o binary:logistic para clasificación)
### Parámetros de regularización:

- gamma: umbral para hacer split basado en la reducción de error de hacer el nuevo split.
- alpha: regularización para los pesos de las hojas. Un valor más alto genera una mayor regularización.
- lambda: similar alpha pero para la sintonia fina.
### Otros parámetros
- Verbosite: si muestra mensajes de info,aviso,error,debug

In [323]:
# Regressor
xg_reg = xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = 0.95, learning_rate = 0.42,
                max_depth = 6, alpha = 20, n_estimators = 300,verbosity=1)

In [317]:
xg_reg.fit(X_train,y_train)

[12:05:23] WARNING: src/objective/regression_obj.cu:144: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(alpha=20, base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=0.95, gamma=0,
       importance_type='gain', learning_rate=0.42, max_delta_step=0,
       max_depth=6, min_child_weight=1, missing=None, n_estimators=300,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=None, subsample=1, verbosity=1)

In [ ]:
preds = xg_reg.predict(X_test)

In [316]:
mse = mean_squared_error(y_test, preds)
print("mean_squared_error: "+str(mse))
rmse = np.sqrt(mse)
print("RMSE: %f" % (rmse))
print("Nico: 3400688504.3380723")
#RMSE con :
#xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = 0.3, learning_rate = 0.1,
#                max_depth = 5, alpha = 10, n_estimators = 10)regresor lineal,
#                colsample_bytree=0.3, learning_rat=0.1,max_depth=1-,n_estimators=10

mean_squared_error: 2.584968880796633e+18
RMSE: 1607783841.440333
Nico: 3400688504.3380723


# Otros casos

In [318]:
train_auc3.head()

,ref_hash,source_id,secs_to_next,hs_to_next,ref_type,secs_to_next_mean,date_int,next_date_int
0,2125435237741841,3,212.0,0.058889,1,40927.75,1555571926350778,1555572138798585
1,2125435237741841,1,4394.0,1.220556,1,40927.75,1555572138798585,1555576532924509
2,2125435237741841,3,1162.0,0.322778,1,40927.75,1555576532924509,1555577694872365
3,2125435237741841,3,42804.0,11.890000,1,40927.75,1555577694872365,1555620498888273
4,2125435237741841,3,62841.0,17.455833,1,40927.75,1555620498888273,1555683339599201


In [319]:
#Separamos X y Y
X2 = train_auc3.loc[:,train_auc3.columns != 'secs_to_next']
y2 = train_auc3.loc[:,train_auc3.columns == 'secs_to_next']
y2.head()

,secs_to_next
0,212.0
1,4394.0
2,1162.0
3,42804.0
4,62841.0


In [321]:
#Filas y columnas
train_auc3.shape,X2.shape,y2.shape

((60778, 8), (60778, 7), (60778, 1))

In [322]:
X_train2,X_test2,y_train2,y_test2 = train_test_split(X2,y2,test_size=0.3,random_state=1)

In [324]:
# Regressor
xg_reg = xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = 0.95, learning_rate = 0.42,
                max_depth = 6, alpha = 20, n_estimators = 300,verbosity=1)

In [325]:
xg_reg.fit(X_train2,y_train2)
preds2 = xg_reg.predict(X_test2)
mse = mean_squared_error(y_test2, preds2)
print("mean_squared_error: "+str(mse))
rmse = np.sqrt(mse)
print("RMSE: %f" % (rmse))
print("Nico: 3400688504.3380723")

[12:19:31] WARNING: src/objective/regression_obj.cu:144: reg:linear is now deprecated in favor of reg:squarederror.
mean_squared_error: 3454587.5162069383
RMSE: 1858.652070
Nico: 3400688504.3380723


In [326]:
xg_reg.feature_importances_

array([1.2281933e-04, 9.5269657e-05, 2.7305067e-01, 9.2463866e-05,
       3.3848044e-02, 1.3531029e-01, 5.5748045e-01], dtype=float32)